In [17]:
import pandas as pd, numpy as np, random
import json
from pathlib import Path

raw = pd.read_csv("../data/processed/registry/fixtures/2025-2026/fixture_calendar.csv")
print("raw is_home counts:", raw["is_home"].value_counts(dropna=False))



raw is_home counts: is_home
0    380
1    380
Name: count, dtype: int64


In [18]:
# Home/away ID alignment must hold row-wise
bad = raw[
    ((raw["is_home"]==1) & ((raw["home_id"]!=raw["team_id"]) | (raw["away_id"]!=raw["opponent_id"]))) |
    ((raw["is_home"]==0) & ((raw["home_id"]!=raw["opponent_id"]) | (raw["away_id"]!=raw["team_id"])))
]
assert bad.empty, "home/away ID misalignment detected"

# Venue sanity: if is_home==1 then team_code should equal 'home' shortcode (when team_code known)
chk = raw[raw["team"].notna()]
assert (chk.loc[chk["is_home"]==1, "team"].str.upper()
          .eq(chk.loc[chk["is_home"]==1, "home"].str.upper())).all(), "venue/code disagreement"


In [19]:
cal = pd.read_csv("../data/processed/registry/fixtures/2025-2026/fixture_calendar.csv")

# 1) For each fpl_id, do you really have exactly one home row and one away row by flags?
cal.groupby("fpl_id")["is_home"].agg(["count","sum"]).query("count!=2 or (sum!=1 and sum.notna())").head(10)

# 2) Are side-ids consistent with flags?
bad = cal[((cal["is_home"]==1) & (cal["team_id"].str.lower()!=cal["home_id"].str.lower()))|
          ((cal["is_home"]==0) & (cal["team_id"].str.lower()!=cal["away_id"].str.lower()))]
print(len(bad), "flag/id mismatches")
bad[["fpl_id","team","is_home","team_id","home_id","away_id","gw_orig"]].head(10)

# 3) Does any (season,fpl_id) appear with multiple distinct home_id/away_id pairs?
pairs = (cal.groupby("fpl_id")[["home_id","away_id"]]
           .nunique().rename(columns={"home_id":"n_home_id","away_id":"n_away_id"}))
pairs.query("n_home_id>1 or n_away_id>1").head(10)

0 flag/id mismatches


,n_home_id,n_away_id
fpl_id,,
